In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import scanpy as sc

import glob
import os
import scrna
import h5py

from micron2 import load_as_anndata
from micron2 import cluster_leiden_cu

import tqdm.auto as tqdm

from matplotlib import pyplot as plt
from matplotlib import rcParams
import time

from micron2.data import staining_border_nonzero

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import rcParams
import seaborn as sns

In [3]:
toss_1 = [f'TMA1_reg{x}_v5' for x in [2,3,4,6,11,16,17,21,22,23,24,27]]
toss_2 = [f'TMA2_reg{x}_v5' for x in [2,10,21,22,26,34]]
toss_3 = [f'TMA3_reg{x}_v5' for x in [1,2,3,7,8,14,15,19,20,21,23,25,32,33]]
toss_patterns = toss_1 + toss_2 + toss_3
def maybe_toss(x):
    for p in toss_patterns:
        if p in x:
            return True
    return False

In [4]:
datasets = !ls -lha /storage/codex/preprocessed_data/*Bladder*/*_v5.hdf5 | awk '{print $9}'
datasets = [d for d in datasets if not maybe_toss(d)]
print(len(datasets))
datasets

76


['/storage/codex/preprocessed_data/210226_Bladder_TMA1_reg10/210226_Bladder_TMA1_reg10_v5.hdf5',
 '/storage/codex/preprocessed_data/210226_Bladder_TMA1_reg1/210226_Bladder_TMA1_reg1_v5.hdf5',
 '/storage/codex/preprocessed_data/210226_Bladder_TMA1_reg12/210226_Bladder_TMA1_reg12_v5.hdf5',
 '/storage/codex/preprocessed_data/210226_Bladder_TMA1_reg13/210226_Bladder_TMA1_reg13_v5.hdf5',
 '/storage/codex/preprocessed_data/210226_Bladder_TMA1_reg14/210226_Bladder_TMA1_reg14_v5.hdf5',
 '/storage/codex/preprocessed_data/210226_Bladder_TMA1_reg15/210226_Bladder_TMA1_reg15_v5.hdf5',
 '/storage/codex/preprocessed_data/210226_Bladder_TMA1_reg18/210226_Bladder_TMA1_reg18_v5.hdf5',
 '/storage/codex/preprocessed_data/210226_Bladder_TMA1_reg19/210226_Bladder_TMA1_reg19_v5.hdf5',
 '/storage/codex/preprocessed_data/210226_Bladder_TMA1_reg20/210226_Bladder_TMA1_reg20_v5.hdf5',
 '/storage/codex/preprocessed_data/210226_Bladder_TMA1_reg25/210226_Bladder_TMA1_reg25_v5.hdf5',
 '/storage/codex/preprocessed_da

In [5]:
ring_channels = ['CD45', 'CD20', 'CD3e', 'CD45RO', 'CD45RA', 'CD8', 'CD4', 'CDH12', 
                 'KRT13', 'KRT17', 'PanCytoK', 'ERBB2']
adatas = []
sample_ids = []
for path in datasets:
    print(path)
    try:
        ad = load_as_anndata(path, 
                             recover_tile_nuclei=False, 
                             as_sparse = False,
                             features_dtype = None
                            )
    except:
        print('failed to load', path)
        continue
    
    # Apply staining border function
    with h5py.File(ad.uns['source_data'], 'r') as h5f:
        ncells = ad.shape[0]
        h5_ncells = h5f['cells/DAPI'].shape[0]
        print(ncells, h5_ncells)
        ring_positive_pct = pd.DataFrame(index=ad.obs_names,
                                         columns=[f'{ch}_ringpct' for ch in ring_channels],
                                         dtype=np.float32
                                        )
        tstart = time.time()
        for i in tqdm.trange(ncells):
            m = h5f['meta/nuclear_masks'][i:i+1,:,:]
            vect = []
            for ch in ring_channels:
                x = h5f[f'cells/{ch}'][i:i+1,:,:]
                v = staining_border_nonzero(x,m)
                vect.append(v)
            ring_positive_pct.loc[ad.obs_names[i],:] = vect
        tend = time.time()
        print(f'elapsed time: {tend-tstart:3.4f}s')
    ad.obs = pd.concat([ad.obs, ring_positive_pct], axis=1)

    
    adatas.append(ad.copy()) 
    s = os.path.splitext(os.path.basename(path))[0]
    sample_ids.append(s)

adata = adatas[0].concatenate(adatas[1:], batch_key='sample_id', batch_categories=sample_ids, 
                              index_unique = '-')
print(sample_ids)
adata.raw = adata


/storage/codex/preprocessed_data/210226_Bladder_TMA1_reg10/210226_Bladder_TMA1_reg10_v5.hdf5
['cell_membrane_stats', 'cell_nuclei_stats', 'cells', 'images', 'meta', 'tile_stats']
['Cell_IDs', 'Tile_IDs', 'bounding_boxes', 'cell_coordinates', 'channel_names', 'membrane_masks', 'nuclear_masks', 'tile_coordinates']
Finished gathering obsm
Loaded nuclear features: (6655, 720)
Loaded membrane features: (6655, 720)
Joined nuclear and membrane features: (6655, 1440)
Joined nuclear and membrane features: 1440
Generating AnnData object
6655 6655


  0%|          | 0/6655 [00:00<?, ?it/s]

elapsed time: 19.4897s
/storage/codex/preprocessed_data/210226_Bladder_TMA1_reg1/210226_Bladder_TMA1_reg1_v5.hdf5
['cell_membrane_stats', 'cell_nuclei_stats', 'cells', 'images', 'meta', 'tile_stats']
['Cell_IDs', 'Tile_IDs', 'bounding_boxes', 'cell_coordinates', 'channel_names', 'membrane_masks', 'nuclear_masks', 'tile_coordinates']
Finished gathering obsm
Loaded nuclear features: (13264, 720)
Loaded membrane features: (13264, 720)
Joined nuclear and membrane features: (13264, 1440)
Joined nuclear and membrane features: 1440
Generating AnnData object
13264 13264


  0%|          | 0/13264 [00:00<?, ?it/s]

elapsed time: 39.1999s
/storage/codex/preprocessed_data/210226_Bladder_TMA1_reg12/210226_Bladder_TMA1_reg12_v5.hdf5
['cell_membrane_stats', 'cell_nuclei_stats', 'cells', 'images', 'meta', 'tile_stats']
['Cell_IDs', 'Tile_IDs', 'bounding_boxes', 'cell_coordinates', 'channel_names', 'membrane_masks', 'nuclear_masks', 'tile_coordinates']
Finished gathering obsm
Loaded nuclear features: (5145, 720)
Loaded membrane features: (5145, 720)
Joined nuclear and membrane features: (5145, 1440)
Joined nuclear and membrane features: 1440
Generating AnnData object
5145 5145


...

elapsed time: 2.9473s
/storage/codex/preprocessed_data/210311_Bladder_TMA3_reg6/210311_Bladder_TMA3_reg6_v5.hdf5
['cell_membrane_stats', 'cell_nuclei_stats', 'cells', 'meta']
['Cell_IDs', 'cell_coordinates', 'channel_names', 'membrane_masks', 'nuclear_masks']
Finished gathering obsm
Loaded nuclear features: (6346, 720)
Loaded membrane features: (6346, 720)
Joined nuclear and membrane features: (6346, 1440)
Joined nuclear and membrane features: 1440
Generating AnnData object
6346 6346


  0%|          | 0/6346 [00:00<?, ?it/s]

elapsed time: 19.0105s
/storage/codex/preprocessed_data/210311_Bladder_TMA3_reg9/210311_Bladder_TMA3_reg9_v5.hdf5
['cell_membrane_stats', 'cell_nuclei_stats', 'cells', 'meta']
['Cell_IDs', 'cell_coordinates', 'channel_names', 'membrane_masks', 'nuclear_masks']
Finished gathering obsm
Loaded nuclear features: (3564, 720)
Loaded membrane features: (3564, 720)
Joined nuclear and membrane features: (3564, 1440)
Joined nuclear and membrane features: 1440
Generating AnnData object
3564 3564


  0%|          | 0/3564 [00:00<?, ?it/s]

elapsed time: 10.5663s
['210226_Bladder_TMA1_reg10_v5', '210226_Bladder_TMA1_reg1_v5', '210226_Bladder_TMA1_reg12_v5', '210226_Bladder_TMA1_reg13_v5', '210226_Bladder_TMA1_reg14_v5', '210226_Bladder_TMA1_reg15_v5', '210226_Bladder_TMA1_reg18_v5', '210226_Bladder_TMA1_reg19_v5', '210226_Bladder_TMA1_reg20_v5', '210226_Bladder_TMA1_reg25_v5', '210226_Bladder_TMA1_reg26_v5', '210226_Bladder_TMA1_reg28_v5', '210226_Bladder_TMA1_reg29_v5', '210226_Bladder_TMA1_reg30_v5', '210226_Bladder_TMA1_reg31_v5', '210226_Bladder_TMA1_reg32_v5', '210226_Bladder_TMA1_reg33_v5', '210226_Bladder_TMA1_reg34_v5', '210226_Bladder_TMA1_reg35_v5', '210226_Bladder_TMA1_reg36_v5', '210226_Bladder_TMA1_reg37_v5', '210226_Bladder_TMA1_reg5_v5', '210226_Bladder_TMA1_reg7_v5', '210226_Bladder_TMA1_reg8_v5', '210226_Bladder_TMA1_reg9_v5', '210308_Bladder_TMA2_reg11_v5', '210308_Bladder_TMA2_reg1_v5', '210308_Bladder_TMA2_reg12_v5', '210308_Bladder_TMA2_reg13_v5', '210308_Bladder_TMA2_reg14_v5', '210308_Bladder_TMA2_r

In [7]:
for ad in adatas:
    print(ad.shape, ad.obs.shape)
    df = ad.obs
    ad.obs = ad.obs.loc[:, ~df.columns.duplicated()]
    
adata = adatas[0].concatenate(adatas[1:], batch_key='sample_id', batch_categories=sample_ids, 
                              index_unique = '-')
print(sample_ids)
adata.raw = adata

(6655, 1440) (6655, 12)
(13264, 1440) (13264, 12)
(5145, 1440) (5145, 12)
(8594, 1440) (8594, 12)
(5255, 1440) (5255, 12)
(7310, 1440) (7310, 12)
(8132, 1440) (8132, 12)
(10050, 1440) (10050, 12)
(10073, 1440) (10073, 12)
(8970, 1440) (8970, 12)
(9045, 1440) (9045, 12)
(14419, 1440) (14419, 12)
(7028, 1440) (7028, 12)
(12983, 1440) (12983, 12)
(8417, 1440) (8417, 12)
(12793, 1440) (12793, 12)
(10912, 1440) (10912, 12)
(4863, 1440) (4863, 12)
(21179, 1440) (21179, 12)
(6673, 1440) (6673, 12)
(8027, 1440) (8027, 12)
(11291, 1440) (11291, 12)
(8769, 1440) (8769, 12)
(11675, 1440) (11675, 12)
(4938, 1440) (4938, 12)
(8274, 1440) (8274, 12)
(8688, 1440) (8688, 12)
(9705, 1440) (9705, 12)
(7066, 1440) (7066, 12)
(5400, 1440) (5400, 12)
(1495, 1440) (1495, 12)
(6844, 1440) (6844, 12)
(10743, 1440) (10743, 12)
(10835, 1440) (10835, 12)
(8301, 1440) (8301, 12)
(639, 1440) (639, 12)
(7901, 1440) (7901, 12)
(8554, 1440) (8554, 12)
(7703, 1440) (7703, 12)
(8274, 1440) (8274, 12)
(16028, 1440) (160

In [8]:
sample_id_printing = ['\n'.join(x.split('_')[2:4]) for x in adata.obs.sample_id]
adata.obs['sample_id_printing'] = sample_id_printing
adata.uns['channels'] = ad.uns['channels']

In [9]:
adata

AnnData object with n_obs × n_vars = 636350 × 1440
    obs: 'CD45_ringpct', 'CD20_ringpct', 'CD3e_ringpct', 'CD45RO_ringpct', 'CD45RA_ringpct', 'CD8_ringpct', 'CD4_ringpct', 'CDH12_ringpct', 'KRT13_ringpct', 'KRT17_ringpct', 'PanCytoK_ringpct', 'ERBB2_ringpct', 'sample_id', 'sample_id_printing'
    uns: 'channels'
    obsm: 'coordinates'

In [11]:
adata.write("/storage/codex/datasets_v1/bladder_merged.h5ad")

... storing 'sample_id_printing' as categorical
